# 6.0 Building the Evaluation Statistical Function #
## For Brownlow Predictor Project ##

This notebook builds evaluation statistical function for AFL Brownlow Predictor H2H version

**Author: `Lang (Ron) Chen` 2023.2-2023.12**

---

In [19]:
import pandas as pd
from collections import defaultdict as dd
import numpy as np

In [2]:
test_data = pd.read_parquet('../data/curated/modelling/test_data.parquet')

In [4]:
obs_y = test_data[['player1', 'player2', 'Brownlow Votes', 'game_id']] # get observed data into useable format

In [27]:
pred_y = test_data[['player1', 'player2', 'game_id']]
pred_y['Predicted Brownlow Votes'] = np.random.normal(size = len(pred_y)) # get predicted data into useable format

/var/folders/xg/8w_3dndd6l5c3n99vd7vd3f40000gn/T/ipykernel_30535/3847147734.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_y['Predicted Brownlow Votes'] = np.random.normal(size = len(pred_y))


In [43]:
brownlow_accuracy_metric = 0 # overall metric

for game_id in set(obs_y['game_id']): # every game

    # initialise tallies
    obs_leaderboard = dd(int)
    pred_leaderboard = dd(int)
    
    # get H2H for one game
    obs_game_data = obs_y[obs_y['game_id'] == game_id]
    pred_game_data = pred_y[pred_y['game_id'] == game_id] 

    # for every head to head, get data
    for row in obs_game_data.iterrows(): # first recover actual Brownlow votes
        if row[1]['Brownlow Votes'] > 0:
            obs_leaderboard[row[1]['player1']] += 1
            obs_leaderboard[row[1]['player2']] -= 1

        elif row[1]['Brownlow Votes'] < 0:
            obs_leaderboard[row[1]['player1']] -= 1
            obs_leaderboard[row[1]['player2']] += 1
    
    for row in pred_game_data.iterrows(): # then get predicted votes
        if row[1]['Predicted Brownlow Votes'] > 0:
            pred_leaderboard[row[1]['player1']] += 1
            pred_leaderboard[row[1]['player2']] -= 1

        elif row[1]['Predicted Brownlow Votes'] < 0:
            pred_leaderboard[row[1]['player1']] -= 1
            pred_leaderboard[row[1]['player2']] += 1
    
    # turn leaderboard into a useable list
    obs_leaderboard = list(obs_leaderboard.items())
    obs_leaderboard.sort(key = lambda x:x[1],  reverse= True)

    pred_leaderboard = list(pred_leaderboard.items())
    pred_leaderboard.sort(key = lambda x:x[1],  reverse= True)

    # finally calculate the statistics of model performance for this game
    for i in range(3):
        if obs_leaderboard[0][0] in pred_leaderboard[i][0]:
            brownlow_accuracy_metric += (3-i) # if 3, add 3; if 2, add 2; if 1, add 1
        
        if obs_leaderboard[1][0] in pred_leaderboard[i][0]:
            brownlow_accuracy_metric += 2-abs(i-2) # if 3, add 1; if 2, add 2; if 1, add 1
    
        if obs_leaderboard[2][0] in pred_leaderboard[i][0]:
            brownlow_accuracy_metric += (i+1)/3 # if 1/3, add 1; if 2/3, add 2; if 1, add 1    

In [46]:
brownlow_accuracy_metric/len(set(obs_y['game_id']))/6

0.06394129979035638

In [ ]:
def get_brownlow_accuracy_metric(obs_y, pred_y):

    """ Helper to evaluate Brownlow Accuracy Metrics """

    brownlow_accuracy_metric = 0 # overall metric

    for game_id in set(obs_y['game_id']): # every game

        # initialise tallies
        obs_leaderboard = dd(int)
        pred_leaderboard = dd(int)
        
        # get H2H for one game
        obs_game_data = obs_y[obs_y['game_id'] == game_id]
        pred_game_data = pred_y[pred_y['game_id'] == game_id] 

        # for every head to head, get data
        for row in obs_game_data.iterrows(): # first recover actual Brownlow votes
            if row[1]['Brownlow Votes'] > 0:
                obs_leaderboard[row[1]['player1']] += 1
                obs_leaderboard[row[1]['player2']] -= 1

            elif row[1]['Brownlow Votes'] < 0:
                obs_leaderboard[row[1]['player1']] -= 1
                obs_leaderboard[row[1]['player2']] += 1
        
        for row in pred_game_data.iterrows(): # then get predicted votes
            if row[1]['Predicted Brownlow Votes'] > 0:
                pred_leaderboard[row[1]['player1']] += 1
                pred_leaderboard[row[1]['player2']] -= 1

            elif row[1]['Predicted Brownlow Votes'] < 0:
                pred_leaderboard[row[1]['player1']] -= 1
                pred_leaderboard[row[1]['player2']] += 1
        
        # turn leaderboard into a useable list
        obs_leaderboard = list(obs_leaderboard.items())
        obs_leaderboard.sort(key = lambda x:x[1],  reverse= True)

        pred_leaderboard = list(pred_leaderboard.items())
        pred_leaderboard.sort(key = lambda x:x[1],  reverse= True)

        # finally calculate the statistics of model performance for this game
        for i in range(3):
            if obs_leaderboard[0][0] in pred_leaderboard[i][0]:
                brownlow_accuracy_metric += (3-i) # if 3, add 3; if 2, add 2; if 1, add 1
            
            if obs_leaderboard[1][0] in pred_leaderboard[i][0]:
                brownlow_accuracy_metric += 2-abs(i-2) # if 3, add 1; if 2, add 2; if 1, add 1
        
            if obs_leaderboard[2][0] in pred_leaderboard[i][0]:
                brownlow_accuracy_metric += (i+1)/3 # if 1/3, add 1; if 2/3, add 2; if 1, add 1   
        
    return brownlow_accuracy_metric/(len(set(obs_y['game_id']))*6) # scale it by game, and also by max score each game could have acquired